In [122]:
import pandas as pd
import sklearn as sk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [93]:
# Lê o arquivo CSV em um DataFrame do pandas
df = pd.read_csv('pre-processed.csv',sep=';')

In [94]:
df.drop('index',inplace=True,axis=1)
df.head()

,label,preprocessed_news
0,0.0,katia abreu diz vai colocar expulsao moldura n...
1,0.0,ray peita bolsonaro conservador fake entrevist...
2,0.0,reinaldo azevedo desmascarado policia federal ...
3,0.0,relatorio assustador bndes mostra dinheiro pub...
4,0.0,radialista americano fala sobre pt vendem ilus...


In [95]:
df.isnull().sum()

label                1
preprocessed_news    1
dtype: int64

In [96]:
df = df.dropna()
df.isnull().sum()

label                0
preprocessed_news    0
dtype: int64

In [97]:
df[df['label']== 0].shape

(3600, 2)

In [98]:
df[df['label']== 1].shape

(3600, 2)

In [99]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [100]:
df['preprocessed_news'] = df['preprocessed_news'].apply(wordopt)

In [101]:
x = df['preprocessed_news']
y = df['label']

In [102]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [104]:
from sklearn.naive_bayes import MultinomialNB

In [105]:
NB = MultinomialNB()
NB.fit(xv_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True, force_alpha='warn')

In [124]:
pred_nb=NB.predict(xv_test)

In [125]:
NB.score(xv_test, y_test)


0.6277777777777778

In [126]:
print(classification_report(y_test, pred_nb))

              precision    recall  f1-score   support

         0.0       0.99      0.25      0.40       711
         1.0       0.58      1.00      0.73       729

    accuracy                           0.63      1440
   macro avg       0.79      0.62      0.56      1440
weighted avg       0.78      0.63      0.57      1440



In [108]:
from sklearn.linear_model import LogisticRegression

In [109]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [110]:
pred_lr=LR.predict(xv_test)

In [111]:
LR.score(xv_test, y_test)

0.9527777777777777

In [123]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95       711
         1.0       0.97      0.94      0.95       729

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440



In [112]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # caso as stopwords não estejam já baixadas

# define as stopwords em português
stopwords_port = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vande\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_NB = NB.predict(new_xv_test)
   

    return print("\n\nLR Prediction: {} \nNB Prediction: {}".format(output_lable(pred_LR[0]),  
                                                                    output_lable(pred_NB[0])))

In [120]:
news = str(input())
palavras = news.split()
palavras_sem_stopwords = [palavra for palavra in palavras if palavra.lower() not in stopwords_port]
texto_sem_stopwords = ' '.join(palavras_sem_stopwords)
manual_testing(texto_sem_stopwords)



LR Prediction: Fake News 
NB Prediction: Not A Fake News


# BAG OF WORDS

In [77]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [78]:
NB = MultinomialNB()
NB.fit(xv_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True, force_alpha='warn')

In [79]:
pred_NB=NB.predict(xv_test)

In [80]:
NB.score(xv_test, y_test)

0.8375

In [81]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
pred_lr=LR.predict(xv_test)

In [83]:
LR.score(xv_test, y_test)

0.9611111111111111